In [66]:
#@title ドライブをマウント
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [67]:
#@title データを保存するディレクトリのパスを指定
#@markdown ランタイムに接続しドライブを後、コピーしてくる<br>
#@markdown 後ろの / は追加するので、追記は不要<br>

cwd = "/content/drive/MyDrive/RNA-seq/Outputs/Hypoxia2" #@param {type:"string"}
cwd = cwd + "/"

# 必要なモジュールをインポートする
import pandas as pd
import numpy as np
from google.colab import files

# カレントディレクトリの変更
import os
os.makedirs(cwd , exist_ok = True)
os.chdir(cwd)
currentdir = os.getcwd()

In [68]:
#@title 実験名を定義
exp_name = "Hypoxia2" #@param {type:"string"}
exp_name = exp_name + "_"

In [69]:
#@title 平均行を追加したTPMデータのパスを定義
df_tpm_path =  "/content/drive/MyDrive/RNA-seq/Outputs/Hypoxia2/Hypoxia2_avetpm.csv" #@param {type:"string"}
df_tpm = pd.read_csv(df_tpm_path , index_col = 0)
df_tpm

,product,N_DC_1,N_DC_2,N_DC_3,N_DC_ave,H_DC_1,H_DC_2,H_DC_3,H_DC_ave
Geneid,,,,,,,,,
ENSG00000000003,TSPAN6,35.429129,30.044254,37.929150,34.467511,32.847463,29.934867,40.241104,34.341144
ENSG00000000005,TNMD,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
ENSG00000000419,DPM1,127.006436,116.304474,144.631756,129.314222,140.908156,146.288098,153.640571,146.945609
ENSG00000000457,SCYL3,6.785430,5.965235,6.699545,6.483403,6.202960,7.992398,6.595434,6.930264
ENSG00000000460,C1orf112,3.220708,6.265905,4.937333,4.807982,2.634051,3.643118,2.929043,3.068737
...,...,...,...,...,...,...,...,...,...
ENSG00000285990,AL589743.7,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
ENSG00000285991,AL355312.5,0.044119,0.000000,0.036128,0.026749,0.000000,0.000000,0.000000,0.000000
ENSG00000285992,AC120036.5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝<br>
GlycoMaple用データの作成

In [70]:
#@title glycan-related_genes.csv の読み込み
df_glyco_path = "/content/drive/MyDrive/RNAseq/SourceFiles/glycan-related_genes.csv" #@param {type:"string"}
df_glyco = pd.read_csv(df_glyco_path)
# print(df_glyco)

In [71]:
#@title GlycoMaple用データの抽出
df_glyco_data = pd.merge(df_glyco , df_tpm)
# print(df_glyco_data)
df_glyco_data.to_csv(exp_name + "for_GlycoMaple.csv" , sep = ",")

＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝<br>
MetaScape用データの作成

In [72]:
#@title データの確認
print(list(df_tpm))

['product', 'N_DC_1', 'N_DC_2', 'N_DC_3', 'N_DC_ave', 'H_DC_1', 'H_DC_2', 'H_DC_3', 'H_DC_ave']


In [73]:
#@title 平均列を比較し、発現変動遺伝子(DEG)を抽出
#@markdown コントロール群の平均列が何列目かを入力
Cont_ave_column = "5" #@param {type:"string"}
Cont_num = int(Cont_ave_column) - 1
#@markdown テスト群の平均列が何列目かを入力
Test_ave_column = "9" #@param {type:"string"}
Test_num = int(Test_ave_column) - 1

df_ave = df_tpm.iloc[:, [Cont_num , Test_num]]

#@markdown 新しい列「DEG」を追加し(コントロール+1)/(テスト+1)の値を計算<br>
#@markdown 増減の理解しやすさのためにlog2の行も追加<br>
#@markdown 列「DEG」の値に基づいてソート

df_ave.loc[: , "DEG"] = (df_ave.iloc[:,1].values + 1) / (df_ave.iloc[:,0].values + 1)
df_ave.loc[: , "DEG_log2"] = np.log2(df_ave.loc[: , "DEG"])
df_ave.sort_values("DEG" , axis = 0 , ascending=False , inplace = True)
df_ave

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1781: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item_labels[indexer[info_axis]]] = value
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentati

,N_DC_ave,H_DC_ave,DEG,DEG_log2
Geneid,,,,
ENSG00000162723,0.991114,17.712421,9.397967,3.232349
ENSG00000092758,0.291194,10.354470,8.793777,3.136483
ENSG00000139219,7.898142,67.122027,7.655759,2.936545
ENSG00000165507,8.344801,66.513454,7.224708,2.852939
ENSG00000159167,1.659201,17.808791,7.073098,2.822342
...,...,...,...,...
ENSG00000211825,10.417129,1.483541,0.217528,-2.200729
ENSG00000266995,9.838155,1.151661,0.198527,-2.332596
ENSG00000252755,4.106081,0.000000,0.195845,-2.352217


In [74]:
#@title 発現変動遺伝子を決定する
#@markdown 発現変動とする基準値を入力(X倍, 1/X倍を基準とする時のX)
DEG_sta = "2" #@param {type:"string"}
sta = float(DEG_sta)
sta = np.log2(sta)

#@markdown 発現変動遺伝子を増加と減少に分けてそれぞれcsvで保存する

df_DEG_up = df_ave[df_ave["DEG_log2"] >= sta]
df_DEG_down = df_ave[df_ave["DEG_log2"] <= -sta]

df_DEG_up.to_csv(exp_name + "発現" + DEG_sta + "倍増加遺伝子.csv" , sep = ",")
df_DEG_down.to_csv(exp_name + "発現" + DEG_sta + "分の1倍減少遺伝子.csv" , sep = ",")

In [75]:
#@title 発現変動値を「実験名_DEGavetpm.csv」ファイルに保存
df_ave = df_ave.loc[: , ["DEG" , "DEG_log2"]]
df_ave.reset_index(inplace=True)
df_tpm.reset_index(inplace=True)
df_tpm_deg = pd.merge(df_tpm , df_ave , how = "inner")
# df_tpm_deg = df_tpm_deg.drop("index" , axis = 1)
df_tpm_deg = df_tpm_deg.set_index(["Geneid" , "product"])
df_tpm_deg.to_csv(exp_name + "DEGavetpm.csv" , sep = ",")
df_tpm_deg

,,N_DC_1,N_DC_2,N_DC_3,N_DC_ave,H_DC_1,H_DC_2,H_DC_3,H_DC_ave,DEG,DEG_log2
Geneid,product,,,,,,,,,,
ENSG00000000003,TSPAN6,35.429129,30.044254,37.929150,34.467511,32.847463,29.934867,40.241104,34.341144,0.996437,-0.005149
ENSG00000000005,TNMD,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
ENSG00000000419,DPM1,127.006436,116.304474,144.631756,129.314222,140.908156,146.288098,153.640571,146.945609,1.135299,0.183072
ENSG00000000457,SCYL3,6.785430,5.965235,6.699545,6.483403,6.202960,7.992398,6.595434,6.930264,1.059714,0.083674
ENSG00000000460,C1orf112,3.220708,6.265905,4.937333,4.807982,2.634051,3.643118,2.929043,3.068737,0.700542,-0.513456
...,...,...,...,...,...,...,...,...,...,...,...
ENSG00000285990,AL589743.7,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
ENSG00000285991,AL355312.5,0.044119,0.000000,0.036128,0.026749,0.000000,0.000000,0.000000,0.000000,0.973948,-0.038084
ENSG00000285992,AC120036.5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
